In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from __future__ import print_function
from util_functions import process_files

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [6]:
batch_size = 64
num_classes = 2
epochs = 10

# load the data
x_train, y_train, f_train = process_files(
                                    dataset='training', 
                                    features=['Mel', 'Mel_deltas'], 
                                    shape='stacked')

x_test, y_test, f_test = process_files(
                                    dataset='test', 
                                    features=['Mel', 'Mel_deltas'], 
                                    shape='stacked')

# input image dimensions
input_h = x_train.shape[1] #Height
input_w = x_train.shape[2] #Width
input_d = x_train.shape[3] #Depth

#Reshaping to feed to network
x_train = x_train.reshape(x_train.shape[0], input_h, input_w, input_d)
x_test = x_test.reshape(x_test.shape[0], input_h, input_w, input_d)
input_shape = (input_h, input_w, input_d)

#Making them floats for TF
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# iterate over different hyperparameters to finetune them
learning_rates = [0.1, 0.01, 0.001, 0.0001]
dropout_rates = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

for lr in learning_rates:
    for dr in dropout_rates:

        print('learning rate: ' + str(lr) + ', dropout: ' + str(dr) + '\n')        
        
        # give unique name to the model
        model_name = 'lr_' + str(lr).replace('.', '_') + '_dr_' + str(dr).replace('.', '_')
        
        model = Sequential()
        model.add(Conv2D(80, kernel_size=(57, 6),
                         activation='relu',
                         input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(80, kernel_size=(1, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(8, 7), strides=(8, 7)))
        model.add(Dropout(dr))
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(dr))
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(dr))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(lr=lr),
                      metrics=['accuracy'])

        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=2,
                  validation_split=0.1)
        score = model.evaluate(x_test, y_test, verbose=0)
        
        # serialize model to JSON
        model_json = model.to_json()
        with open('models_keras/sound_classifier_' + model_name + '.json', 'w') as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5        
        model.save_weights('models_keras/sound_classifier_' + model_name + '.h5')
        
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])        
        
        print('----------------------------------------------------\n')    



learning rate: 0.1, dropout: 0

Train on 1282 samples, validate on 143 samples
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# PREDICTIONS FROM SAVED MODEL
from keras.models import model_from_json

model_name = 'sound_classifier_lr_0_1_dr_0'

# load json and create model
json_file = open('models_keras/' + model_name + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights('models_keras/' + model_name + '.h5')

# create predictions
score = loaded_model.predict(x_test)